#Installation

In [14]:
!pip install keras-nightly
!pip uninstall -y tensorflow
%tensorflow_version 1.x
!pip install --upgrade h5py==2.10.0
!pip install -U scikit-image==0.16.2
!pip install tensorflow 
import sys
sys.path.append("/content/Mask_RCNN")
%matplotlib inline

Found existing installation: tensorflow 2.8.0
Uninstalling tensorflow-2.8.0:
  Successfully uninstalled tensorflow-2.8.0


ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\shiva\AppData\Local\Programs\Python\Python310\lib\site-packages\pip\_internal\cli\base_command.py", line 167, in exc_logging_wrapper
    status = run_func(*args)
  File "C:\Users\shiva\AppData\Local\Programs\Python\Python310\lib\site-packages\pip\_internal\commands\uninstall.py", line 102, in run
    uninstall_pathset.commit()
  File "C:\Users\shiva\AppData\Local\Programs\Python\Python310\lib\site-packages\pip\_internal\req\req_uninstall.py", line 420, in commit
    self._moved_paths.commit()
  File "C:\Users\shiva\AppData\Local\Programs\Python\Python310\lib\site-packages\pip\_internal\req\req_uninstall.py", line 273, in commit
    save_dir.cleanup()
  File "C:\Users\shiva\AppData\Local\Programs\Python\Python310\lib\site-packages\pip\_internal\utils\temp_dir.py", line 173, in cleanup
    rmtree(self._path)
  File "C:\Users\shiva\AppData\Local\Programs\Python\Python310\lib\site-packages\pip\_vendor\tenacity\__init__.p

In [15]:
!pip install keras


In [16]:
import tensorflow as tf
from tensorflow import keras

In [17]:
import os
import sys
import random
import numpy as np
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt

#Dataset Preparation

In [19]:
base_dir = "G:\Git\ML_Project\Dataset\ML_Project\VOC2007"
annotations = "G:\Git\ML_Project\Dataset\ML_Project\VOC2007\Annotations"
images = "G:\Git\ML_Project\Dataset\ML_Project\VOC2007\JPEGImages"

#Create Custom Dataset field for loading our Pascal Dataset

In [20]:
from mrcnn.utils import Dataset

class PascalDataset(Dataset):
    '''
    mainly three methods, load_datset, load_masks and image_reference needs to be modified according to our dataset
    '''

    def load_dataset(self, txt_file ,dataset_dir):

      #adding all classes from Pascal Dataset
        self.add_class("dataset",1,'aeroplane')
        self.add_class("dataset",2,'bicycle')
        self.add_class("dataset",3,'bird')
        self.add_class("dataset",4,'boat')
        self.add_class("dataset",5,'bottle')
        self.add_class("dataset",6,'bus')
        self.add_class("dataset",7,'car')
        self.add_class("dataset",8,'cat')
        self.add_class("dataset",9,'chair')
        self.add_class("dataset",10,'cow')
        self.add_class("dataset",11,'diningtable')
        self.add_class("dataset",12,'dog')
        self.add_class("dataset",13,'horse')
        self.add_class("dataset",14,'motorbike')
        self.add_class("dataset",15,'person')
        self.add_class("dataset",16,'pottedplant')
        self.add_class("dataset",17,'sheep')
        self.add_class("dataset",18,'sofa')
        self.add_class("dataset",19,'train')
        self.add_class("dataset",20,'tvmonitor')

        # define data locations
        images_dir = dataset_dir + '/JPEGImages/'
        annotations_dir = dataset_dir + '/Annotations/'

        with open(txt_file) as f:
          all_images = f.read().split('\n')

        # Split the dataset, if train, get 90%, else 10%
        # len_images = len(os.listdir(images_dir))
        # if is_train == "train":
        #     img_range = [int(len_images / 9), len_images]
        # else:
        #     img_range = [0, int(len_images / 9)]

        # find all images
        # for i in range(img_range[0],img_range[1]):
        # for filename in os.listdir(images_dir):
        for image_id in all_images:
            
            # extract image id
            # image_id = filename
            # skip bad images
            
            img_path = images_dir + image_id + '.jpg'
            ann_path = annotations_dir + image_id + '.xml'
            
            # check if is_train is true or false based on that decide if it'll be in train_dataset
            
            # add to dataset
            self.add_image('dataset', image_id=image_id, path=img_path, annotation=ann_path)

    def extract_boxes(self,filename):
      '''
      extracting all the bounding boxes avaialable in the given image
      '''
      # load and parse the file
      tree = ET.parse(filename)
      # get the root of the document
      root = tree.getroot()
      # extract each bounding box
      objects = list() #list of tuples (name,box_coors)
      for obj in root.findall('.//object'):
          xmin = int(obj.find('./bndbox/xmin').text)
          ymin = int(obj.find('./bndbox/ymin').text)
          xmax = int(obj.find('./bndbox/xmax').text)
          ymax = int(obj.find('./bndbox/ymax').text)
          name = obj.find('./name').text
          coors = [xmin, ymin, xmax, ymax]
          objects.append((name,coors))
      # extract image dimensions
      width = int(root.find('.//size/width').text)
      height = int(root.find('.//size/height').text)
      return objects, width, height
  
    #load mask for an image
    def load_mask(self, image_id):
        # get details of image
        info = self.image_info[image_id]
        
        # define box file location
        path = info['annotation']
        
        # load XML
        objects, w, h = self.extract_boxes(path)
        
        # create one array for all masks, each on a different channel
        masks = np.zeros([h, w, len(objects)], dtype='uint8')
        
        # create masks
        class_ids = list()
        for i in range(len(objects)):
            name, box = objects[i]
            row_s, row_e = box[1], box[3]
            col_s, col_e = box[0], box[2]
            masks[row_s:row_e, col_s:col_e, i] = 1
            class_ids.append(self.class_names.index(name))
        return masks, np.asarray(class_ids, dtype='int32')
    
    #load an Image referance
    def image_referance(self, image_id):
        info = self.image_info[image_id]
        return info['path']

ModuleNotFoundError: No module named 'mrcnn'